# Image Classification with DNN

## DATASETS:
(a) Carbonic Anhydrase II (ChEMBL205), a protein lyase,  
(b) Cyclin-dependent kinase 2 (CHEMBL301), a protein kinase,  
(c) ether-a-go-go-related gene potassium channel 1 (HERG) (CHEMBL240), a voltage-gated ion channel,  
(d) Dopamine D4 receptor (CHEMBL219), a monoamine GPCR,  
(e) Coagulation factor X (CHEMBL244), a serine protease,  
(f) Cannabinoid CB1 receptor (CHEMBL218), a lipid-like GPCR and  
(g) Cytochrome P450 19A1 (CHEMBL1978), a cytochrome P450.  
The activity classes were selected based on data availability and as representatives of therapeutically important target classes or as anti-targets.

In [1]:
!nvidia-smi

Thu Mar 17 09:42:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
|  0%   54C    P8    16W / 240W |    549MiB /  8116MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
# Import
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import sys
import os
sys.path.append('/usr/local/lib/python3.7/site-packages/')
#from rdkit import Chem
#from rdkit.Chem import AllChem

In [3]:
dataset = 'CHEMBL205'

In [5]:
path = Path('../dataset/13321_2017_226_MOESM1_ESM/')
df = pd.read_csv(path/dataset/f'{dataset}_train_valid.csv')

In [6]:
list(path.iterdir())

[PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL205'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/.ipynb_checkpoints'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL301'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL218'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL219'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL244'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/mol_images'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL1978'),
 PosixPath('../dataset/13321_2017_226_MOESM1_ESM/CHEMBL240')]

In [7]:
DATA = path/'mol_images'

In [8]:
IMAGES = DATA/dataset

# Create train validation split

In [30]:
from sklearn.model_selection import train_test_split

In [9]:
df.is_valid.value_counts()

False    14352
True      3589
Name: is_valid, dtype: int64

In [11]:
len(df[df.is_valid == True]) / len(df[df.is_valid == False])

0.2500696767001115

# Create dataloader

In [15]:
from fastai.vision.all import *

In [16]:
df['Image'] = df['CID'] + '.png'
df.head()

,CID,SMILES,Activity,is_valid,Image
0,CHEMBL188002,S(=O)(=O)(N)c1cc(N/C(/S)=N\c2cc(C(=O)[O-])c(cc2)C=2c3c(OC4=CC(=O)C=CC=24)cc(O)cc3)ccc1,1,False,CHEMBL188002.png
1,CHEMBL364127,Clc1ccc(cc1)C(=O)NC1Cc2cc(S(=O)(=O)N)ccc2C1,1,False,CHEMBL364127.png
2,CHEMBL1683469,S(=O)(=O)(N)c1ccc(cc1)CNS(=O)(=O)CC12CCC(CC1=O)C2(C)C,1,False,CHEMBL1683469.png
3,CHEMBL52564,Oc1ccccc1\C=C\C(=O)[O-],1,False,CHEMBL52564.png
4,CHEMBL21427,OB(O)c1ccc(OC)cc1,1,False,CHEMBL21427.png


In [17]:
images = df['CID'] + ".png"
images.head()

0     CHEMBL188002.png
1     CHEMBL364127.png
2    CHEMBL1683469.png
3      CHEMBL52564.png
4      CHEMBL21427.png
Name: CID, dtype: object

In [18]:
# No flips, some rotation and zoom. Normalize to imagenet_stats as
# we're using pretrained models.
batch_tfms = [*aug_transforms(mult=1.0, do_flip=False, flip_vert=False, 
               max_rotate=20.0, min_zoom=1.0, max_zoom=1.1, max_lighting=0., 
               max_warp=0., p_affine=0.75, p_lighting=0.75, xtra_tfms=None, 
               size=224, mode='bilinear', 
               pad_mode='border', align_corners=True, 
               batch=False, min_scale=1.0), 
              Normalize.from_stats(*imagenet_stats)]

In [19]:
db = DataBlock(
    blocks = (ImageBlock(), CategoryBlock()), 
    get_x=ColReader('Image', pref=IMAGES),
    get_y=ColReader('Activity'), 
    splitter=ColSplitter('is_valid'),
    item_tfms=None, 
    batch_tfms=None,
    )

Oversmaple instead of using class weights to test custom loss function

In [20]:
def val_cols_and_oversample(df, test_size=0.25, random_state=42, oversample=True):
    x_train,x_val = train_test_split(df.index, test_size=test_size, stratify=df['Activity'], 
                                     random_state=random_state)
    
    df.loc[x_train, 'is_valid']=False
    df.loc[x_val, 'is_valid']=True
    # Oversample is_valid=False columns
    if oversample:
        df = pd.concat([*[df.loc[(df.is_valid==False) & (df.Activity==1)]]*10, 
                        *[df.loc[(df.is_valid==False) & (df.Activity==0)]], 
                        df.loc[df.is_valid==True]], 
                       ignore_index=True)
    return df


In [27]:
def get_df(dataset, random_state=42, test_size=0.25, oversample=True):
    df = pd.read_csv(path/dataset/f'{dataset}_cl.csv')
    df = val_cols_and_oversample(df, test_size=test_size, random_state=random_state, oversample=oversample)
    df['Image'] = df['CID'] + '.png'
    return df

In [28]:
def get_df_with_clusters(dataset, n_clusters=100, random_state=42, test_size=0.25):
    df = pd.read_csv(path/dataset/f'{dataset}_cl_ECFP_1024_with_{n_clusters}_clusters.csv')
    values = df.Cluster.value_counts(ascending=True)
    values = values[values < 2].index
    for i in list(values):
        df = df.drop(df[df.Cluster == i].index)
    df.reset_index(drop=True, inplace=True)
    x_train, x_valid = train_test_split(df.index, test_size=test_size, random_state=random_state, stratify=df['Cluster'])
    df.loc[x_train, 'is_valid']=False
    df.loc[x_valid, 'is_valid']=True
    df['Image'] = df['Name'] + '.png'
    return df

In [31]:
df = get_df(dataset)
df.head()

,CID,SMILES,Activity,is_valid,Image
0,CHEMBL364127,Clc1ccc(cc1)C(=O)NC1Cc2cc(S(=O)(=O)N)ccc2C1,1,False,CHEMBL364127.png
1,CHEMBL1683469,S(=O)(=O)(N)c1ccc(cc1)CNS(=O)(=O)CC12CCC(CC1=O)C2(C)C,1,False,CHEMBL1683469.png
2,CHEMBL52564,Oc1ccccc1\C=C\C(=O)[O-],1,False,CHEMBL52564.png
3,CHEMBL21427,OB(O)c1ccc(OC)cc1,1,False,CHEMBL21427.png
4,CHEMBL1414,O(C)c1cc(cc(OC)c1O)C(=O)[O-],1,False,CHEMBL1414.png


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28948 entries, 0 to 28947
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   CID       28948 non-null  object
 1   SMILES    28948 non-null  object
 2   Activity  28948 non-null  int64 
 3   is_valid  28948 non-null  object
 4   Image     28948 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.1+ MB


In [40]:
IMAGES = DATA/f'{dataset}_cl'
IMAGES

Path('../dataset/13321_2017_226_MOESM1_ESM/mol_images/CHEMBL205_cl')

In [43]:
def get_dls(dataset, bs=64, clustering=False, random_state=42):
    IMAGES = DATA/f'{dataset}_cl'
    if clustering:
        df = get_df_with_clusters(dataset, n_clusters=100)
    else: 
        df = get_df(dataset, oversample=True)
    db = DataBlock(
        blocks = (ImageBlock(), CategoryBlock()), 
        get_x=ColReader('Image', pref=IMAGES),
        get_y=ColReader('Activity'), 
        splitter=ColSplitter('is_valid'),
        item_tfms=None, 
        batch_tfms=None,
        )
    dls = db.dataloaders(df, bs=bs, shuffle_train=True, random_state=42)
    return dls, df

In [44]:
dls, df = get_dls(dataset, clustering=False)

Could not do one pass in your dataloader, there is something wrong in it


In [46]:
dls.show_batch()

RuntimeError: Could not infer dtype of PILImage

In [ ]:
dls.show_batch(unique=True)

# Create class weights

In [28]:
import torch

In [29]:
TARGET = 'Activity'

In [30]:
train_df = df[df.is_valid == False]

In [31]:
class_count_df = train_df.groupby(TARGET).count()
class_count_df

,CID,SMILES,is_valid,Image
Activity,,,,
0,12232,12232,12232,12232
1,12230,12230,12230,12230


In [32]:
n_0, n_1 = class_count_df.iloc[0, 0], class_count_df.iloc[1, 0]

In [33]:
w_0 = (n_0 + n_1) / (2.0 * n_0)
w_1 = (n_0 + n_1) / (2.0 * n_1)
w_0, w_1

(0.9999182472204055, 1.0000817661488144)

In [34]:
torch.cuda.device(0)

In [35]:
torch.cuda.is_available()

False

In [36]:
torch.cuda.get_device_name(0)

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
class_weights=torch.FloatTensor([w_0, w_1]).cuda()

In [ ]:
class_weights=torch.FloatTensor([w_0, w_1]).cuda()

In [ ]:
loss_func = CrossEntropyLossFlat(weight=class_weights)

In [ ]:
def get_loss_func(df):
    TARGET = 'Activity'
    class_count_df = df.groupby(TARGET).count()
    n_0, n_1 = class_count_df.iloc[0, 0], class_count_df.iloc[1, 0]
    w_0 = (n_0 + n_1) / (2.0 * n_0)
    w_1 = (n_0 + n_1) / (2.0 * n_1)
    class_weights=torch.FloatTensor([w_0, w_1]).cuda()
    loss_func = CrossEntropyLossFlat(weight=class_weights)
    return loss_func

In [ ]:
loss_func=get_loss_func(df)

# Train CNN model

In [ ]:
from sklearn.metrics import auc,roc_auc_score,recall_score,precision_score,f1_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
import seaborn as sns
from utils.losses import * 

In [ ]:
from fastai.callback.hook import *

In [ ]:
def train_model(dls, arch=resnet18, loss_func=None, metric = None, epochs=16, wd=None, lr=None):
    
    
    print(f'Architechture: {arch}')
    #print(f'Untrained epochs: freeze_epochs={freeze_epochs}')
    print(f'Trained epochs: epochs={epochs}')
    print(f'Weight decay: wd={wd}')
    if metric == None: 
        learn = cnn_learner(dls, arch=arch, pretrained=True, 
                            wd=wd,
                            loss_func=loss_func,
                            metrics=[accuracy, 
                            F1Score(), 
                            Precision(), 
                            Recall(), 
                            RocAucBinary(), 
                            MatthewsCorrCoef()])
    else: 
        learn = cnn_learner(dls, arch=arch, pretrained=True, 
                                wd=wd,
                                loss_func=loss_func,
                                metrics=[accuracy, 
                                F1Score(), 
                                Precision(), 
                                Recall(), 
                                RocAucBinary(), 
                                MatthewsCorrCoef(), 
                                metric])
    
    if lr == None: 
        print(f'Finding learning rate...')
        lr_min, lr_steep = learn.lr_find(suggestions=True, show_plot=False)
        print(f'Training model with learning rate: {lr_min}')
        lr = lr_min
    else: 
        print(f'Training model with learning rate: {lr}')
    learn.fine_tune(epochs, lr)
    
    return learn

In [ ]:
# Train model on resnet18
learn = train_model(dls,loss_func=None, epochs=5, lr=3e-3, wd=0.0002)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(k=50)

# Test for metric with cost

In [ ]:
cost = [100, 1]

In [ ]:
n_0, n_1 = class_count_df.iloc[0, 0], class_count_df.iloc[1, 0]

In [ ]:
w_0 = (n_0 + n_1) / (2.0 * n_0)
w_1 = (n_0 + n_1) / (2.0 * n_1)
w_0 = w_0 * cost[0]
w_1 = w_1 * cost[1]
w_0, w_1

In [ ]:
class_weights=torch.FloatTensor([w_0, w_1]).cuda()

In [ ]:
decision_loss = nn.CrossEntropyLoss(weight=class_weights)
type(decision_loss)

In [ ]:
type(F1Score())

In [ ]:
# Train model on resnet18
learn = train_model(dls,loss_func=loss_func, metric=decision_loss, epochs=1, lr=3e-3, wd=0.002)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(k=5)

# Cost-sensitive loss function



There's a simple idea behind a Cost-Sensitive Loss: we impose a different penalty to different kind of errors. For illustration purposes, imagine you have a three-class classification problem and you wish to impose a greater penalty to errors in misclassifying class 2 as class 0, but you don't care about the other errors.*

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
n_classes = 2

In [ ]:
cs_loss = CostSensitiveLoss(n_classes, normalization='sigmoid')

In [ ]:
print(cs_loss.M)

In [ ]:
M = np.array([[0,10],
              [1,0]], dtype=float)
M /= M.max()
M = torch.from_numpy(M)
M = M.to(torch.float32)
cs_loss.M = M


In [ ]:
print(cs_loss.M)

In [ ]:
# Train model on resnet18
learn = train_model(dls,loss_func=cs_loss, metric=cs_loss, epochs=1, lr=3e-3, wd=0.002)

In [ ]:
show_confusion_matrix(learn, dls)

**Results**: Just training for 1 epoch, number of False Positives went down, number of False Negatives went up (as expected), but number of True positives also went up which is bad. 

# Regularized cost-sensitive loss function

In our experiments (and elsewhere) we found out that simply using a CS loss leads to lots of trouble in terms of CNNs staying at local minima where they will predict a trivial configuration (all the time the same category), which seems to be very satisfying for this kind of losses. For this reason, we recommend using this as a regularizer for other standard classification losses (just a weighted sum of both, with a weighing parameter `lambd`). \
In our implementation we provide a wrapper for doing this, where you specify a `base_loss` and the regularization parameter `lambd`:


In [ ]:
n_classes = 2 
base_loss = 'ce'
lambd = 10
cs_regularized_loss = CostSensitiveRegularizedLoss(n_classes=n_classes, base_loss=base_loss, lambd=lambd)

In [ ]:
print(cs_regularized_loss.M)

In [ ]:
M = np.array([[0,10],
              [1,0]], dtype=float)
M /= M.max()
M = torch.from_numpy(M)
M = M.to(torch.float32)
cs_regularized_loss.M = M

In [ ]:
print(cs_regularized_loss.M)

In [ ]:
# Train model on resnet18
learn = train_model(dls,loss_func=cs_regularized_loss, metric=cs_regularized_criterion, epochs=1, lr=3e-3, wd=0.002)

In [ ]:
preds, y = learn.get_preds(dl=dls.valid)

In [ ]:
preds

In [ ]:
def show_confusion_matrix(learn, dl):
    preds, y = learn.get_preds(dl=dl)
    pred = preds.argmax(axis=1)
    cm = confusion_matrix(y, pred)
    f = sns.heatmap(cm, annot=True, fmt='d')
    plt.show()
    return f

In [ ]:
show_confusion_matrix(learn, dls.valid)

**Results**:  False positives went drastically down, and true positives up compared to just using cost-sensitive loss, precision is higher then using regular cross entropy loss so could be a bit random. 

### Train for 10 epochs: 

In [ ]:
learn = train_model(dls,loss_func=cs_regularized_loss, metric=cs_regularized_criterion, epochs=10, lr=3e-3, wd=0.002)

In [ ]:
show_confusion_matrix (learn, dls)

# Compare regularized cost-sensitive loss functions

In [ ]:
from utils.losses import *

Compare: Cross entropy (CE), CE with Label Smoothing (LS), CE with Gaussian LS

In [ ]:
losses = ['ce', 'focal_loss', 'ls', 'gls']

In [ ]:
M = np.array([[0,20],
            [1,0]], dtype=float)
M /= M.max()
M = torch.from_numpy(M)
M = M.to(torch.float32)

In [ ]:
def train_model_with_loss(base_loss='ce', n_classes = 2, lambd = 10):
    loss = CostSensitiveRegularizedLoss(n_classes=n_classes, base_loss=base_loss, lambd=lambd)
    loss.M = M
    learn = train_model(dls,loss_func=loss, epochs=5, lr=3e-3, wd=0.002)
    cm = show_confusion_matrix(learn, dls.valid)
    return learn, cm

In [ ]:
lambd = 2
for l in losses:
    if l == 'gls':
        lambd = 2
    print(f'Training model with base loss function {l} and lambd: {lambd}')
    base_loss = l
    train_model_with_loss(base_loss=base_loss, lambd=lambd)

# Test different lambdas

Lamdas is used to choose how much you will weight the cost-sensitive loss functions vs cross-entropy

In [ ]:
%%script echo skipping
lambdas = [0, 0.1, 1, 10]
lambdas

In [ ]:
n_classes = 2
base_loss = 'ce'
for l in lambdas:
    train_model_with_loss(base_loss=base_loss, lambd=l)

In [ ]:
n_classes = 2
base_loss = 'focal_loss'
for l in lambdas:
    train_model_with_loss(base_loss=base_loss, lambd=l)

In [ ]:
%%script false --no-raise-error # save
n_classes = 2
base_loss = 'ls'
for l in lambdas:
    train_model_with_loss(base_loss=base_loss, lambd=l)

In [ ]:
%%script false --no-raise-error
n_classes = 2
base_loss = 'gls'
for l in lambdas:
    print(f'Training model with base loss function {base_loss} and lambda: {l}')
    train_model_with_loss(base_loss=base_loss, lambd=l)